---
title: Tracking Changes
date: 2023-11-30 
authors:
  - name: Sébastien Boisgérault
    email: Sebastien.Boisgerault@minesparis.psl.eu
    url: https://github.com/boisgera
    affiliations:
      - institution: Mines Paris - PSL University
        department: Institut des Transformation Numériques (ITN)
github: boisgera
license: CC-BY-4.0
open_access: true
---

In order to understand how `.tldr` files are structured, we can add a new graphical objects, change some if their properties, etc. and each time we modify the document, analyze the corresponding evolution of the file.

In this notebook, we develop some tooling to help us track such changes.

## Text comparison

We define two similar versions of the "zen of Python":

In [2]:
zen_1 = """The Zen of Python, by Tim Peters

Beautiful is better than ugly.
Explicit is better than implicit.
Simple is better than complex.
Complex is better than complicated.
Flat is better than nested.
Sparse is better than dense.
Readability counts.
Special cases aren't special enough to break the rules.
Errors should never pass silently.
In the face of ambiguity, refuse the temptation to guess.
There should be one obvious way to do it.
Although that way may not be obvious at first.
Now is better than never.
Although never is often better than right now.
If the implementation is hard to explain, it's a bad idea.
If the implementation is easy to explain, it is a good idea.
"""

In [3]:
zen_2 = """\
The Zen of Python, by Tim Peters

Beautiful is better than ugly.
Explicit is better than implicit.
Simple is better than complex.
Complex is better than complicated.
Flat is better than nested.
Sparse is better than dense.
Readability counts.
Special cases aren't special enough to break the rules.
Although practicality beats purity.
Errors should never pass silently.
Unless explicitly silenced.
In the face of ambiguity, refuse the temptation to guess.
There should be one-- and preferably only one --obvious way to do it.
Although that way may not be obvious at first unless you're Dutch.
Now is better than never.
Although never is often better than *right* now.
If the implementation is hard to explain, it's a bad idea.
If the implementation is easy to explain, it may be a good idea.
Namespaces are one honking great idea -- let's do more of those!
"""

```{exercise}
 1. Transform `zen_1` and `zen_2` into list of lines.
 2. Use the [`difflib`](https://docs.python.org/3/library/difflib.html) module of the Python standard library to [`compare`](https://docs.python.org/3/library/difflib.html#difflib.Differ.compare) the two sequences.
 3. Make a text out of the output of compare and print it.
 4. Interpret the result and list the differences between both versions of the zen of Python.
```

In [5]:
"""1"""
zen_1_lines = zen_1.split('\n')
zen_2_lines = zen_2.split('\n')

for line in zen_1_lines:
    print(line)

for line in zen_2_lines:
    print(line)

"""2"""
from difflib import SequenceMatcher

matcher = SequenceMatcher(None, zen_1_lines, zen_2_lines)

similarity_ratio = matcher.ratio()

print("Rapport de similarité :", similarity_ratio)

"""3 et 4"""

for tag, i1, i2, j1, j2 in matcher.get_opcodes():
    if tag == 'replace':
        print('Différence trouvée entre zen 11[{}:{}] et zen2[{}:{}]'.format(i1, i2, j1, j2))
        print('  Ancienne valeur (zen1) :', zen_1_lines[i1:i2])
        print('  Nouvelle valeur (zen2) :', zen_2_lines[j1:j2])
    elif tag == 'delete':
        print('Élément(s) supprimé(s) de zen 1[{}:{}]'.format(i1, i2))
        print('  Valeur(s) supprimée(s) :', zen_1_lines[i1:i2])
    elif tag == 'insert':
        print('Élément(s) inséré(s) dans zen 2[{}:{}]'.format(j1, j2))
        print('  Valeur(s) insérée(s) :', zen_2_lines[j1:j2])
    elif tag == 'equal':
        print('zen1[{}:{}] est égale à zen2[{}:{}]'.format(i1, i2, j1, j2))


The Zen of Python, by Tim Peters

Beautiful is better than ugly.
Explicit is better than implicit.
Simple is better than complex.
Complex is better than complicated.
Flat is better than nested.
Sparse is better than dense.
Readability counts.
Special cases aren't special enough to break the rules.
Errors should never pass silently.
In the face of ambiguity, refuse the temptation to guess.
There should be one obvious way to do it.
Although that way may not be obvious at first.
Now is better than never.
Although never is often better than right now.
If the implementation is hard to explain, it's a bad idea.
If the implementation is easy to explain, it is a good idea.

The Zen of Python, by Tim Peters

Beautiful is better than ugly.
Explicit is better than implicit.
Simple is better than complex.
Complex is better than complicated.
Flat is better than nested.
Sparse is better than dense.
Readability counts.
Special cases aren't special enough to break the rules.
Although practicality beat

We can make our job easier if we use HTML instead of plain text to visualise the differences between the two texts.


```{exercise}
  1. Use the [HtmlDiff](https://docs.python.org/3/library/difflib.html#difflib.HtmlDiff) class of difflib to produce a `diff.html` file that represents this difference in a HTML document.
  2. Use the [webbrowser](https://docs.python.org/3/library/webbrowser.html) module of the standard library to open it!
  3. Define a `display_diff_text` function that takes two arguments `text_1` and `text_2` and automates steps 1. and 2.
```

In [15]:
"""1"""

from difflib import HtmlDiff

html_diff = HtmlDiff()
html_content = html_diff.make_file(zen_1_lines, zen_2_lines)

html_file = open("diff.html", "w") 
html_file.write(html_content)

"""2"""

import webbrowser
webbrowser.open_new_tab("diff.html")

"""3"""

def display_diff_text(text_1, text_2):
    html_diff = HtmlDiff()
    html_content = html_diff.make_file(text_1, text_2)
    with open("diff_fct.html", "w") as html_file:
        html_file.write(html_content)
    html_file_path = "diff_fct.html"
    webbrowser.open_new_tab("diff_fct.html")
    
display_diff_text(zen_1_lines, zen_2_lines)


## Comparison of JSON documents

````{exercise} Comparison of dictionnaries

 1. Create a `display_diff` function that takes two Python objects, converts them to strings then leverages `display_diff_text` to display the difference in a browser.

 2. Consider the 3 dictionaries defined by
    ```python
    d1 = {k:k+1 for k in range(100)}
    d2 = d1.copy(); d2[50] = 50
    d3 = {k:k+1 for k in range(99, -1, -1)}
    ```
    `d1` and `d2` have a slight difference and `d1` and `d2` are equal.
    Does your `display_diff` function make easy to spot where the difference is in the first case when it compares `d1` and `d2`?
    Does it make easy to see that `d1` and `d3` are equal?

  3. Investigate the [`pprint`](https://docs.python.org/3/library/pprint.html) module standard library ; use it to improve the behavior of `display_text_diff` in the two cases considered in the previous question.

````
 

In [24]:
"""1"""

def display_diff(object_1, object_2):
    text_1 = str(object_1)
    text_2 = str(object_2)

    html_content = display_diff_text(text_1, text_2)
    webbrowser.open_new_tab("diff_fct.html")


"""2"""

d1 = {k:k+1 for k in range(100)}
d2 = d1.copy(); d2[50] = 50
d3 = {k:k+1 for k in range(99, -1, -1)}

display_diff(d1,d2)
display_diff(d1,d3)


""" """

"""3"""

import pprint

def new_display_diff_text(text1, text2):
    html_diff = HtmlDiff()

    pprint_text1 = pprint.pformat(text1, width=80)
    pprint_text2 = pprint.pformat(text2, width=80)
    
    return html_diff.make_file(pprint_text1.splitlines(), pprint_text2.splitlines())



```{exercise} tldraw documents comparator
Implement a function `tldraw_diff` that takes as argument two filenames that refer to tldraw documents and display their differences in the browser.
```

In [26]:

def tldraw_diff(file1, file2):
    content_1 = load_tldraw_file(file1)
    content_2 = load_tldraw_file(file2)

    html_content = new_display_diff_text(content_1, content_2)

    html_file = open("diff.html", "w")
    html_file.write(html_content)
        
    webbrowser.open_new_tab("diff.html")

def load_tldraw_file(filename):
    file = open(filename, 'r')
    return file.read()